# Start EVA Server

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

In [2]:
## Install EVA package if needed
%pip install "evadb[udf]" --quiet

import os
import time
import subprocess


def shell(command):
    try:
        result = subprocess.run(command, capture_output=False, text=True)
        message = result.stdout
        message += result.stderr
        print(message)
    except Exception as e:
        message = "exception"
        print(message)

def stop_eva_server():
    # Kill any process listening on EVA's standard port
    shell('pgrep eva')
    shell('kill -9 $(lsof -ti:5432)')

def launch_eva_server():

    # Stop EVA server if it is running
    stop_eva_server()

    os.environ['GPU_DEVICES'] = '0'
    shell("rm nohup.out")

    # Start EVA server
    shell("nohup eva_server &")
    shell("cat nohup.out")

    # Wait for server to start
    time.sleep(10)

def connect_to_server():

    from eva.server.db_api import connect
    %pip install nest_asyncio --quiet
    import nest_asyncio
    nest_asyncio.apply()

    # Connect client with server
    connection = connect(host = '127.0.0.1', port = 5432) 
    cursor = connection.cursor()

    return cursor

# Launch server
launch_eva_server()